# Извлечение информации

([Задание для семинара](#scrollTo=_r19P5FYuGk2&line=1&uniqifier=1), 
[Домашнее задание](#scrollTo=PbBbqFUeQPcS&line=1&uniqifier=1))


Задачей извлечения информации является получение структурированного знания из набора неструктурированных текстов. 

## Домашнее задание (6 баллов)

Обучите кластеризатор триплетов открытого извлечения информации. Кластером будет считаться некоторое семантическое отношение, представленное в тексте разными формами написания. Поля ``subject``, ``object``, ``text_predicate`` можно использовать при кластеризации; поле ``wikidata_property`` используется для оценки качества извлечения отношений.

В предложенных данных объединены таблицы набора триплетов из базы знаний wikidata и набора триплетов, извлеченных из сырых текстов с помощью OpenIE. Соответствие между триплетами установлено напрямую по совпадению пар субъект-объект без семантической фильтрации предикатов и потому может быть ложным (например, если человек родился в том же городе, в котором умер - пары субъект-объект не дублируются) или неявным (человек был эвакуирован из города, в котором родился и жил, но пары субъект-объект опять же не дублируются).

In [2]:
! wget -q https://www.dropbox.com/s/ajd5wl6hu7bxen4/ie_exploration.csv

In [3]:
import pandas as pd

data = pd.read_csv('ie_exploration.csv')

In [4]:
data.head(1)

,subject,object,wikidata_property,text_predicate
0,Artvin Province,Turkey,country,is province in


In [5]:
data[data.wikidata_property == 'place of birth'].text_predicate.value_counts()

was born in                     131
is in                             5
died in                           5
grew up in                        2
born in                           2
is filmmaker from                 1
was popular actor in              1
former mayor of                   1
was tuscan painter from           1
was native of                     1
earl of                           1
coach from                        1
was german astronomer from        1
is former footballer from         1
was born on                       1
was born child in                 1
was australian journalist of      1
was born at                       1
was mayor of                      1
was born second in                1
is japanese composer from         1
was politician from               1
lived in                          1
recalled to                       1
was baptized in                   1
went home to                      1
native of                         1
returned to                 

In [6]:
data.wikidata_property.unique()

array(['country', 'located in the administrative territorial entity',
       'place of birth', 'occupation', 'place of death', 'creator',
       'genre', 'instrument', 'production company', 'named after',
       'mouth of the watercourse', 'publisher', 'performer', 'developer',
       'position played on team / speciality', 'director', 'child',
       'author', 'country of citizenship', 'instance of', 'screenwriter',
       'cause of death', 'country of origin', 'parent taxon', 'composer',
       'crosses', 'founded by', 'original language of film or TV show',
       'place of burial', 'record label', 'producer',
       'located in time zone', 'location', 'conflict', 'family',
       'film editor', 'sex or gender'], dtype=object)

In [7]:
print("Число wikidata-отношений в данных:", len(data.wikidata_property.unique()))

Число wikidata-отношений в данных: 37


Оценка качества кластеризации (= извлечения отношений) на примере случайных меток кластеров:

In [8]:
from sklearn.metrics.cluster import adjusted_mutual_info_score
import numpy as np

random_clusters = np.random.randint(0, 38, size=data.shape[0])
adjusted_mutual_info_score(data.wikidata_property, random_clusters)

0.004852614840143447

Baseline: скажем, что предикаты открытого извлечения информации - это и есть семантические отношения.

In [9]:
adjusted_mutual_info_score(data.wikidata_property, data.text_predicate)

0.5520540914769483

In [469]:
texts = []

for index, row in data.iterrows():
    sent = []
    #sent += row['subject'].split(' ')
    #sent += row['object'].split(' ')
    sent += row['text_predicate'].split(' ')
    texts.append(sent)

In [470]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=texts, vector_size=100, window=2, min_count=1, seed=42)

In [471]:
Vec_s = np.zeros((len(data), 100), np.float32)
Vec_o = np.zeros((len(data), 100), np.float32)
Vec_tp = np.zeros((len(data), 100), np.float32)
for i in range(len(data)):
    #Vec_s[i] = sum(model.wv[data['subject'][i].split(' ')])/len(data['subject'][i].split(' '))
    #Vec_o[i] = sum(model.wv[data['object'][i].split(' ')])/len(data['object'][i].split(' '))
    Vec_tp[i] = sum(model.wv[data['text_predicate'][i].split(' ')])/len(data['text_predicate'][i].split(' '))

In [472]:
#data_vec_SOTP = pd.concat([pd.DataFrame(Vec_s), pd.DataFrame(Vec_o), pd.DataFrame(Vec_tp)], axis=1)
data_vec_TP = pd.DataFrame(Vec_tp)

In [473]:
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans

clustering = SpectralClustering(n_clusters=20, affinity='nearest_neighbors', n_neighbors=12, random_state=42).fit(data_vec_SOTP)
kmeans = KMeans(n_clusters=37, n_init=50, random_state=42).fit(data_vec_TP)

In [474]:
adjusted_mutual_info_score(data.wikidata_property, clustering.labels_)

0.4997600681075202

In [475]:
adjusted_mutual_info_score(data.wikidata_property, kmeans.labels_)

0.5258194292625538

Решения в форме ноутбуков, где последняя ячейка содержит ответ с текстовым комментарием, присылайте на mipttextanalysis20@gmail.com (ссылкой на colab.research или файлом). 

Решения без штрафа (8 баллов максимум) принимаются до 12.00 20 октября. Всё, что присылается не в срок, оценивается из максимума 4 балла. Работу, присланную в срок и оцененную не менее чем на 4 балла, можно доделать и досдать на максимум один раз.

Пожалуйста, указывайте фамилию в названии блокнота. Комментарий к решению принимается на русском или английском языке.

### Комментарий

Для векторизации слов я использовал word2vec, стоп-слова не удалял, тк без них качество снижалось. Для последовательности слов вычислялось среднее для каждого элемента вектора.
Для кластеризации я использовал SpectralClustering и KMeans из sklearn.cluster, наилучшие результаты adjusted_mutual_info_score приблизительно равны 52,5%, что несколько ниже Baseline. Для KMeans наилучший скор получается при использовании только данных text_predicate, SpectralClustering лучше всего работает на всех доступных данных, но при этом с количеством кластеров 20. Возможно существует более эффективная векторизация, но при текущем результате baseline выигрывает.